In [ ]:
import os
import time
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score, f1_score,
    roc_curve, auc, precision_recall_curve, average_precision_score
)
import matplotlib.pyplot as plt

# ---------------- Config ----------------
MIRNA_SIM_F = "mirna_similarity.npy"
GENE_SIM_F = "gene_similarity.npy"
INTERACTION_F = "interaction.npy"
MODEL_WEIGHTS_F = "mirtmc_final_weights.pt" # File lưu trọng số mô hình cuối cùng

USE_CUDA = torch.cuda.is_available()
# Chú ý: Cần kiểm tra device ID có tồn tại không
if USE_CUDA:
    try:
        # Sử dụng device ID 0
        DEVICE = torch.device("cuda:0")
    except RuntimeError:
        # Fallback về CPU nếu cuda:0 không khả dụng
        DEVICE = torch.device("cpu")
else:
    DEVICE = torch.device("cpu")

print("Use CUDA:", USE_CUDA, "Device:", DEVICE)

TEST_SUBSET = False       # chạy full

# ADMM / SVD params  
K_SVD = 500
OVERSAMPLE = 20
N_POWER_ITERS = 2
MU = 1e-3
RHO = 1.5
MAX_ITER = 100
TOL = 1e-4

# CV params
K_FOLD = 5     
LAMBDA_CANDIDATES = [0.001, 0.01, 0.1, 1.0, 10.0]  
TORCH_DTYPE = torch.float32

# CHỈ TÍNH TOP K = 10
TOP_K_RANGE = [10]
# ----------------------------------------
 
def evaluate_ranking(I_pred, test_pos, m, n):
    """Tính AUC và AUPR toàn cục."""
    all_mi_gene_pairs = np.array([[i, j] for i in range(m) for j in range(n)])
    labels = np.zeros(len(all_mi_gene_pairs), dtype=int)
    test_pos_set = set(map(tuple, test_pos))
    
    # Chỉ xét các cặp (miRNA, Gene) có trong ma trận kích thước m x n
    is_mi_gene_pair = (all_mi_gene_pairs[:, 0] < m) & (all_mi_gene_pairs[:, 1] < n)
    valid_pairs = all_mi_gene_pairs[is_mi_gene_pair]

    # Cập nhật labels cho các tương tác dương trong tập test
    for k, (mi, gene) in enumerate(valid_pairs):
        if (mi, gene) in test_pos_set:
            labels[k] = 1
            
    scores = I_pred[valid_pairs[:, 0], valid_pairs[:, 1]]
    
    auc_val = roc_auc_score(labels, scores)
    aupr = average_precision_score(labels, scores)
 
    fpr, tpr, _ = roc_curve(labels, scores)
    precision_pr, recall_pr, _ = precision_recall_curve(labels, scores)

    return auc_val, aupr, fpr, tpr, labels, scores, precision_pr, recall_pr

def calculate_topk_metrics(I_pred, test_pos, m, n, k_range=TOP_K_RANGE):
    """
    Tính Precision@K, Recall@K, F1@K, HR@K, NDCG@K cho K trong k_range.
    Trả về một dictionary lồng nhau: {K: {metric_name: value}}
    """
    test_pos_set = set(map(tuple, test_pos))
    results_by_k = {k: {'P': [], 'R': [], 'F1': [], 'HR': [], 'NDCG': []} for k in k_range}
    
    # Lặp qua TẤT CẢ miRNA (mi)
    for i in range(m):
        scores = I_pred[i, :]
        arg_sort_desc = np.argsort(scores)[::-1]
        
        # 1. Xác định các Gene đích thực sự (relevance) cho miRNA i
        relevant_genes_in_test = [j for j in range(n) if (i, j) in test_pos_set]
        
        # Chỉ tính metrics cho các miRNA có tương tác dương trong tập test
        if not relevant_genes_in_test:
            continue
            
        relevance = np.zeros(n, dtype=int)
        for j in relevant_genes_in_test:
            relevance[j] = 1
        
        num_true_positives = len(relevant_genes_in_test)

        # 2. Lặp qua các ngưỡng K
        for k in k_range:
            top_k_genes_indices = arg_sort_desc[:k]
            
            # Tính Hits và Relevance cho Top K
            hits = [g_idx for g_idx in top_k_genes_indices if g_idx in relevant_genes_in_test]
            num_hits = len(hits)
            
            # --- Precision, Recall, F1 ---
            precision_k = num_hits / k
            recall_k = num_hits / num_true_positives
            f1_k = 2 * precision_k * recall_k / (precision_k + recall_k) if (precision_k + recall_k) > 0 else 0
            
            results_by_k[k]['P'].append(precision_k)
            results_by_k[k]['R'].append(recall_k)
            results_by_k[k]['F1'].append(f1_k)
            
            # --- HR (Hit Ratio) ---
            hr_k = 1 if num_hits >= 1 else 0
            results_by_k[k]['HR'].append(hr_k)

            # --- NDCG ---
            top_k_relevance = np.array([relevance[j] for j in top_k_genes_indices])
            
            # DCG
 
            dcg = np.sum(top_k_relevance / np.log2(np.arange(2, len(top_k_relevance) + 2)))
            
            # IDCG
            ideal_relevance = np.sort(relevance)[::-1][:k]
            idcg = np.sum(ideal_relevance / np.log2(np.arange(2, len(ideal_relevance) + 2)))
            
            ndcg_val = dcg / idcg if idcg > 0 else 0
            results_by_k[k]['NDCG'].append(ndcg_val)

    # 3. Tính giá trị trung bình cho tất cả K
    final_metrics = {}
    for k in k_range:
        final_metrics[k] = {
            'P': np.mean(results_by_k[k]['P']) if results_by_k[k]['P'] else 0,
            'R': np.mean(results_by_k[k]['R']) if results_by_k[k]['R'] else 0,
            'F1': np.mean(results_by_k[k]['F1']) if results_by_k[k]['F1'] else 0,
            'HR': np.mean(results_by_k[k]['HR']) if results_by_k[k]['HR'] else 0,
            'NDCG': np.mean(results_by_k[k]['NDCG']) if results_by_k[k]['NDCG'] else 0,
        }
        
    return final_metrics

def run_evaluation_only(X_completed_t, I, positives, m, n, best_lambda):
 
    
    aucs = []; auprs = []
    all_labels = []; all_scores = []
    topk_fold_metrics = {k: {metric: [] for metric in ['P', 'R', 'F1', 'HR', 'NDCG']} for k in TOP_K_RANGE}

    I_pred_saved = X_completed_t[:m, m:].cpu().numpy()

    print("\nStarting K-Fold Evaluation using Saved Weights...")
    kf_eval = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)

    for fold, (train_idx, test_idx) in enumerate(kf_eval.split(positives)):
        train_pos, test_pos = positives[train_idx], positives[test_idx]
        
        auc, aupr, _, _, labels, scores, _, _ = evaluate_ranking(I_pred_saved, test_pos, m, n)
        topk_results = calculate_topk_metrics(I_pred_saved, test_pos, m, n, k_range=TOP_K_RANGE)

        aucs.append(auc); auprs.append(aupr)
        all_labels.append(labels); all_scores.append(scores)
        
        # Lưu metrics Top K của fold hiện tại
        k_disp = TOP_K_RANGE[0]
        for k in TOP_K_RANGE:
            for metric in ['P', 'R', 'F1', 'HR', 'NDCG']:
                topk_fold_metrics[k][metric].append(topk_results[k][metric])
        
        print(f"--- Fold {fold+1}/{K_FOLD} --- AUC={auc:.4f}, AUPR={aupr:.4f}, P@{k_disp}={topk_results[k_disp]['P']:.4f}, HR@{k_disp}={topk_results[k_disp]['HR']:.4f}, NDCG@{k_disp}={topk_results[k_disp]['NDCG']:.4f}")

    mean_auc, std_auc = np.mean(aucs), np.std(aucs)
    mean_aupr, std_aupr = np.mean(auprs), np.std(auprs)
    
    all_results_row = {'lambda': best_lambda, 'AUC_mean': mean_auc, 'AUC_std': std_auc, 'AUPR_mean': mean_aupr, 'AUPR_std': std_aupr}
    
    # Thêm metrics Top K
    for k in TOP_K_RANGE:
        for metric in ['P', 'R', 'F1', 'HR', 'NDCG']:
            mean_val = np.mean(topk_fold_metrics[k][metric])
            std_val = np.std(topk_fold_metrics[k][metric])
            all_results_row[f'{metric}@{k}_mean'] = mean_val
            all_results_row[f'{metric}@{k}_std'] = std_val
    
    results_df = pd.DataFrame([all_results_row])
 
    final_labels = np.concatenate(all_labels)
    final_scores = np.concatenate(all_scores)
    
    return results_df, best_lambda, final_labels, final_scores

 
def main():
    print("Loading numpy files...")
    # Tải ma trận tương đồng (S_m, S_g) và tương tác (I)
    try:
        S_m = np.load(MIRNA_SIM_F)
        S_g = np.load(GENE_SIM_F)
        I = np.load(INTERACTION_F)
    except FileNotFoundError as e:
        print(f"Error: File not found. Please ensure {MIRNA_SIM_F}, {GENE_SIM_F}, and {INTERACTION_F} are in the same directory.")
        print(f"Missing file: {e.filename}")
        return
        
    m, n = I.shape
    positives = np.argwhere(I == 1)

 
    if not os.path.exists(MODEL_WEIGHTS_F):
         print(f"Error: Model weights file '{MODEL_WEIGHTS_F}' not found. Cannot run evaluation-only mode.")
         print("Please ensure the trained weights file is present or run the full training code first.")
         return

    checkpoint = torch.load(MODEL_WEIGHTS_F, map_location=DEVICE)
    X_completed_t = checkpoint['X_completed'].to(DEVICE, dtype=TORCH_DTYPE)
    best_lambda = checkpoint.get('lambda', 'N/A')
    print(f"Loading final weights from {MODEL_WEIGHTS_F} for evaluation (Best Lambda: {best_lambda})...")
    
    # Thực hiện đánh giá K-Fold  
    results_df, final_lambda, final_labels, final_scores = run_evaluation_only(
        X_completed_t, I, positives, m, n, best_lambda
    )
    
    # Sắp xếp và in bảng kết quả  
    print("\n\nFINAL RESULTS SUMMARY (K=10):")
    
    # Lấy hàng của lambda tốt nhất  
    best_row = results_df.iloc[0]

    print(f"\nAll Results (Mean ± Std for Loaded $\lambda$: {final_lambda}):\n")
    
    summary_data = []
    summary_data.append(('Metric', 'Mean ± Std'))
    summary_data.append(('---', '---'))
    
    # Metrics Global
    summary_data.append(('AUC', f"{best_row['AUC_mean']:.4f} ± {best_row['AUC_std']:.4f}"))
    summary_data.append(('AUPR', f"{best_row['AUPR_mean']:.4f} ± {best_row['AUPR_std']:.4f}"))
    summary_data.append(('---', '---'))
    
    # Metrics Top K (K=10)
    k = 10
    if f'P@{k}_mean' in best_row.index:
        summary_data.append((f'P@{k}', f"{best_row[f'P@{k}_mean']:.4f} ± {best_row[f'P@{k}_std']:.4f}"))
        summary_data.append((f'R@{k}', f"{best_row[f'R@{k}_mean']:.4f} ± {best_row[f'R@{k}_std']:.4f}"))
        summary_data.append((f'F1@{k}', f"{best_row[f'F1@{k}_mean']:.4f} ± {best_row[f'F1@{k}_std']:.4f}"))
        summary_data.append((f'HR@{k}', f"{best_row[f'HR@{k}_mean']:.4f} ± {best_row[f'HR@{k}_std']:.4f}"))
        summary_data.append((f'NDCG@{k}', f"{best_row[f'NDCG@{k}_mean']:.4f} ± {best_row[f'NDCG@{k}_std']:.4f}"))
        
    print(pd.DataFrame(summary_data[2:], columns=summary_data[:2]).to_string(index=False))

if __name__=="__main__":
    main()